In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

print(tf.__version__)

2.0.0-rc1


In [2]:
#Import from file
data = pd.read_csv("data.txt", names=['time', 'value', 'zero'])
#Drop null column
data.drop('zero', axis=1, inplace=True)
#Make time-column start from zero
data['time'] = data['time'].apply(lambda x : x - data['time'][0])
#Print first 5 rows
data.head(5)
#print(data.shape)

FileNotFoundError: [Errno 2] File b'data.txt' does not exist: b'data.txt'

In [ ]:
data.plot(x ='time', y='value', kind='line', figsize=(100,10))

In [ ]:
train = data.iloc[10000:16000]
test = data.iloc[5000:8000]
train.plot(x ='time', y='value', kind='scatter', figsize=(100,10))
test.plot(x ='time', y='value', kind='scatter', figsize=(100,10))

In [ ]:
#Convert dataframe to nparray
train_data = train.to_numpy()#train.values
print(train_data.shape)
#Train on 5000, test on 1000
TRAIN_SPLIT=5000

#Normalize
#train_mean = train_data[:TRAIN_SPLIT].mean()
#train_std = train_data[:TRAIN_SPLIT].std()
#train_data = (train_data-train_mean)/train_std

#train_data = (train_data-train_data.min())/(train_data.max()-train_data.min())

print(train_data.shape)

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    dataset = dataset[:, [1]]
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [ ]:
#Preduict next, using past 80
past_history = 500
future_target = 50

x_train_uni, y_train_uni = univariate_data(train_data, 0, TRAIN_SPLIT, past_history, future_target)
x_val_uni, y_val_uni = univariate_data(train_data, TRAIN_SPLIT, None, past_history, future_target)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target value to predict')
print (y_train_uni[0])

In [ ]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
def create_time_steps(length):
    time_steps = []
    for i in range(-length, 0, 1):
        time_steps.append(i)
    return time_steps

def show_plot(plot_data, delta, title):
    labels = ['History', 'Ground truth', 'Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
          plt.plot(future, plot_data[i], marker[i], markersize=10,
                   label=labels[i])
        else:
          plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt

def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, 0]), label='History')
    plt.plot(np.arange(num_out)/1, np.array(true_future), 'bo',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/1, np.array(prediction), 'ro',
             label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()
    
for x, y in val_univariate.take(1):
    multi_step_plot(x[0], y[0], np.array([0]))

In [ ]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(future_target)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 30

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

In [ ]:
mpl.rcParams['figure.figsize'] = (17, 8)
mpl.rcParams['axes.grid'] = False
#for x, y in val_univariate.take(100):
#    plot = show_plot([x[future_target].numpy(), y[future_target].numpy(), simple_lstm_model.predict(x)[future_target]], future_target, 'Simple LSTM model')
#    plot.show()

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for x, y in val_univariate.take(3): # x is number one, y is function number two
    y_pred = simple_lstm_model.predict(x)[0]
    multi_step_plot(x[0], y[0], y_pred)
    print(mean_absolute_percentage_error(y[0], y_pred))
    # print(tf.keras.losses.MAPE(y[0], y_pred)) Doesnt seem to work as I want it to. Probably means that something is wrong.